# Flight Comptuer

We recorded a lot of data about the flight comptuer itself during the flight.


## System Power Comumption

Power used by each subsystem over time

In [ ]:
from numpy import loadtxt, array, subtract, divide, multiply, median, std, var, average, arange, absolute
from scipy.signal import butter, lfilter, freqz
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

t_0 = 117853569585227

columns = loadtxt("../fc-data/RNHP.csv", delimiter=',', unpack=True)

seqn = columns[0]
timestamp = columns[1]
port_1 = columns[2]
port_2 = columns[3]
port_3 = columns[4]
port_4 = columns[5]
umbilical = columns[6]
port_6 = columns[7]
port_7 = columns[8]
port_8 = columns[9]

timestamp = subtract(timestamp, t_0)
timestamp = divide(timestamp, 1e9)

def lowpass(port):
    # Filter requirements.
    order = 6
    fs = 1000.0       # sample rate, Hz
    cutoff = 25       # desired cutoff frequency of the filter, Hz
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq

    # Get the filter coefficients so we can check its frequency response.
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return lfilter(b, a, port)

port_1_filter = lowpass(port_1)
port_2_filter = lowpass(port_2)
port_3_filter = lowpass(port_3)
port_4_filter = lowpass(port_4)
port_6_filter = lowpass(port_6)
port_7_filter = lowpass(port_7)
port_8_filter = lowpass(port_8)

fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"Avionics System Current Consumption")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.ylabel(r"Current [A]")
ax1.plot(timestamp[100:-100], port_1_filter[100:-100], alpha=0.75, label="Flight Comptuer")
ax1.plot(timestamp[100:-100], port_8_filter[100:-100], alpha=0.75, label="IMU")
ax1.plot(timestamp[100:-100], port_3_filter[100:-100], alpha=0.75, label="GPS")
ax1.plot(timestamp[100:-100], port_7_filter[100:-100], alpha=0.75, label="Roll Control Servo Actuator")
ax1.plot(timestamp[100:-100], port_6_filter[100:-100], alpha=0.75, label="WiFi Power Amp")
ax1.set_xlim([-5,42])
ax1.set_ylim([0,1.5])
ax1.legend(loc=2)
plt.show()

The roll control system comes on at $t=0$ and turns intself off at $t=32$. This does appear to mess with the power draw of the whole system. There is a mysterious jump in the WiFi power draw at $~t=26$.

In [ ]:
columns = loadtxt("../fc-data/RNHH.csv", delimiter=',', unpack=True)

seqn            = columns[ 0]
rnhh_time       = columns[ 1]
batt_temp       = columns[ 2]
TS1_temp        = columns[ 3]
TS2_temp        = columns[ 4]
temprange       = columns[ 5]
voltage         = columns[ 6]
current         = columns[ 7]
avg_current     = columns[ 8]
cell1_v         = columns[ 9]
cell2_v         = columns[10]
cell3_v         = columns[11]
cell4_v         = columns[12]
pack_voltage    = columns[13]
avg_voltage     = columns[14]

rnhh_time = subtract(rnhh_time, t_0)
rnhh_time = divide(rnhh_time, 1e9)


fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"Avionics System Voltage")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.ylabel(r"Voltage [V]")
ax1.plot(rnhh_time, voltage, alpha=0.75, label="System Voltage")
ax1.set_xlim([-5,42])
ax1.set_ylim([14,18])
#ax1.legend(loc=2)
plt.show()

Not much to see here. Two small transients that seem to match the large current draw spikes in the system power.

We can see the average full stack power draw, which is intesting as a data point:

In [ ]:
print "Average Power Draw:  %0.2f W (%0.2f Amps at %0.1f Volts)" % (average(voltage) * -average(current), -average(current), average(voltage))

That's a lot of power! Our system battery is a 4s stack of liPo's rated for ~60 Wh. That's about two hours of autonomous flight. We have umbilical power until the last few minutes of the countdown.

## RasPi Computers

In addition to the main flight computer, we had two Raspberry Pi single-board computers as video recorder/processers. These also took power from the system power supply:

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"Avionics System Current Consumption")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.ylabel(r"Current [A]")
ax1.plot(timestamp[100:-100], port_2_filter[100:-100], alpha=0.75, label="RasPi 1")
ax1.plot(timestamp[100:-100], port_4_filter[100:-100], alpha=0.75, label="RasPi 2")
ax1.set_xlim([-5,42])
ax1.set_ylim([0,1.3])
ax1.legend(loc=2)
plt.show()

Due to a camera failure just before flight, we flew with RasPi 1 powered off.

## Missing Packets

One potential question to ask is there any correlation with the power system and missing data packets. Here is the total power, adding up the current reported from each subsystem, both raw and low pass filtered.

There are some spikes, do they line up at all with the missing data?

In [ ]:
total_power = port_1 + port_2 + port_3 + port_4 + port_6 + port_7 + port_8

# Filter requirements.
order = 6
fs = 1000.0       # sample rate, Hz
cutoff = 15       # desired cutoff frequency of the filter, Hz
nyq = 0.5 * fs
normal_cutoff = cutoff / nyq

# Get the filter coefficients so we can check its frequency response.
b, a = butter(order, normal_cutoff, btype='low', analog=False)
total_power_filter = lfilter(b, a, total_power)


columns = loadtxt("../fc-data/SEQE.csv", delimiter=',', unpack=True)

seqn      = columns[0]
seqe_time = columns[1]
seqe_time = subtract(seqe_time, t_0)
seqe_time = divide(seqe_time, 1e9)

fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"Avionics System Current Consumption")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.ylabel(r"Current [A]")
ax1.plot(timestamp, total_power, 'k-', alpha=0.12, label="Total Power Raw")
ax1.plot(timestamp[100:-100], total_power_filter[100:-100], alpha=0.8, label="Total Power Filtered")

for t in seqe_time[1:]:
    plt.plot((t, t), (-10, 10), 'g-', alpha=0.7)
plt.plot((-5, -5), (-10, 10), 'g-', alpha=0.4, label="Data Gaps")

ax1.set_xlim([-5,42])
ax1.set_ylim([0,5.5])
ax1.legend(loc=2)
plt.show()

Nothing obvious in total power, looking at each individual power draw again:

In [ ]:

fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"Avionics System Current Consumption")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.ylabel(r"Current [A]")
ax1.plot(timestamp[100:-100], port_1_filter[100:-100], alpha=0.75, label="Flight Comptuer")
ax1.plot(timestamp[100:-100], port_8_filter[100:-100], alpha=0.75, label="IMU")
ax1.plot(timestamp[100:-100], port_3_filter[100:-100], alpha=0.75, label="GPS")
ax1.plot(timestamp[100:-100], port_7_filter[100:-100], alpha=0.75, label="Roll Control Servo Actuator")
ax1.plot(timestamp[100:-100], port_6_filter[100:-100], alpha=0.75, label="WiFi Power Amp")
ax1.plot(timestamp[100:-100], port_4_filter[100:-100], alpha=0.75, label="RasPi 2")

for t in seqe_time[1:]:
    plt.plot((t, t), (-10, 10), 'g-', alpha=0.4)

plt.plot((-5, -5), (-10, 10), 'g-', alpha=0.4, label="Data Gaps")

ax1.set_xlim([-5,42])
ax1.set_ylim([0,1.9])
ax1.legend(loc=2)
plt.show()

A mildly plausible correlation in Flight Comptuer power at $t=34.72$. Nothing else stands out.